# 체크포인터를 사용한 상태 관리

### 실습내용

해당 내용은 요즘 ai 에이전트 개발 llM RAG ADK MCP LangChain A2A (저자: 박승규) 에서 첨부하였습니다.

| Saver         | 언제 사용       |
| ------------- | ----------- |
| InMemorySaver | 지금 (연습/테스트) |
| SQLiteSaver   | 로컬 영속 저장    |
| RedisSaver    | 세션 기반 메모리   |
| S3 / DB Saver | 장기 메모리      |


In [2]:
# 필요한 라이브러리 
from typing import Dict, Any
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver  # ① InMemorySaver 임포트
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import json

In [3]:
# 그래프 상태 정의
class MemoryBotState(BaseModel):
    user_message: str = Field(default="", description="사용자 입력 메시지")
    user_name: str = Field(default="", description="사용자 이름")
    user_preferences: Dict[str, Any] = Field(default_factory=dict, description="사용자 선호도")
    response: str = Field(default="", description="최종 응답")
    

In [4]:
llm = ChatOpenAI(model="gpt-5-mini")

In [12]:
#메시지 처리 노드
def process_message(state: MemoryBotState) -> Dict[str, Any]:
    message = state.user_message
    user_name = state.user_name
    preferences = state.user_preferences.copy()
    
    # 시스템 프롬프트
    system_prompt = f"""
당신은 사용자의 정보를 기억하는 메모리 봇입니다.
현재 기억하고 있는 정보:
- 사용자 이름: {user_name if user_name else "모름"}
- 좋아하는 것: {preferences.get("likes", [])}
- 싫어하는 것: {preferences.get("dislikes", [])}

사용자 메시지를 분석하여 다음 JSON 형태로 응답하세요:
{{
  "response": "사용자에게 줄 응답 메시지",
  "new_name": "새로 알게 된 이름 (없으면 null)",
  "new_likes": ["새로 알게 된 좋아하는 것들"],
  "new_dislikes": ["새로 알게 된 싫어하는 것들"]
}}
"""

    messages = [SystemMessage(content=system_prompt), HumanMessage(content = message)]
    
    response = llm.invoke(messages)
    result = json.loads(response.content)
    
    #새로운 정보 없데이트
    if result.get("new_name"):
        user_name = result["new_name"]
    
    if result.get("new_likes"):
        preferences.setdefault("likes", []).extend(result["new_likes"])
    
    if result.get("new_dislikes"):
        preferences.setdefault("dislikes", []).extend(result["new_dislikes"])
    
    bot_response = result.get("response", "죄송해요, 이해하지 못했어요.")
    
    
    return {"response" : bot_response,
            "user_name" : user_name,
            "user_preferences" : preferences
    }
    # result = json.loads(response.content)

In [ ]:
# test_case = MemoryBotState(
#     user_message = "초콜릿이 너무 좋아",
#     user_name="정현두"
# )

# result = process_message(test_case)
# print(result)

RAW AIMessage: content='{\n  "response": "초콜릿을 좋아하시는군요! 기억해둘게요.",\n  "new_name": null,\n  "new_likes": ["초콜릿"],\n  "new_dislikes": []\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 313, 'prompt_tokens': 143, 'total_tokens': 456, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D0en1Z1AaZNX5F8XsjBfHHd09PY3J', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019be38c-122f-7fa2-8293-3b6a52cb5b69-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 143, 'output_tokens': 313, 'total_tokens': 456, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}}
CONTENT: '{\n  "respons

In [13]:
    # 메모리 봇 그래프 생성
def create_memory_bot_graph():
    # InmemorySaver로 자동 메모리 관리
    checkpointer = InMemorySaver()
    
    workflow = StateGraph(MemoryBotState)
    
    workflow.add_node("process_message", process_message)
    
    workflow.add_edge(START, "process_message")
    workflow.add_edge("process_message", END)
    
    #checkpointer와 함께 컴파일
    return workflow.compile(checkpointer=checkpointer)
    

In [15]:
def main():
    print("=== InMemorySaver 메모리 봇 테스트 ===\n")
     
    app = create_memory_bot_graph()
    thread_id = "hyun_123"  # ⑦ thread_id - 세션 식별자
    
    conversations = [
        "안녕하세요!",
        "내 이름은 현두야",
        "김치찜을 좋아해",
        "해삼은 싫어해",
        "내 이름이 뭐였지?",
        "내가 좋아하는 것과 싫어하는 것은?",
    ]
    
    for i, message in enumerate(conversations, 1):
        print(f"[{i}] 사용자: {message}")
        
        # ⑧ InMemorySaver 사용을 위한 config 설정
        config = {"configurable": {"thread_id": thread_id}}
        result = app.invoke({"user_message": message}, config)
        
        print(f"[{i}] 챗봇: {result['response']}")
        print(
            f"메모리: 이름={result.get('user_name', '없음')}, "
            f"좋아하는 것={result.get('user_preferences', {})}\n"
        )        

In [16]:
if __name__ == "__main__":
    main()

=== InMemorySaver 메모리 봇 테스트 ===

[1] 사용자: 안녕하세요!
[1] 챗봇: 안녕하세요! 만나서 반가워요. 저는 당신에 대해 정보를 기억할 수 있어요. 이름이나 좋아하는 것, 싫어하는 것을 알려주시면 저장하겠습니다.
메모리: 이름=, 좋아하는 것={}

[2] 사용자: 내 이름은 현두야
[2] 챗봇: 반가워, 현두야! 이름 알려줘서 고마워.
메모리: 이름=현두, 좋아하는 것={}

[3] 사용자: 김치찜을 좋아해
[3] 챗봇: 김치찜 좋아하시는군요! 기억해둘게요.
메모리: 이름=현두, 좋아하는 것={'likes': ['김치찜']}

[4] 사용자: 해삼은 싫어해
[4] 챗봇: 알겠어요. 해삼을 싫어하는 것으로 기억할게요.
메모리: 이름=현두, 좋아하는 것={'likes': ['김치찜'], 'dislikes': ['해삼']}

[5] 사용자: 내 이름이 뭐였지?
[5] 챗봇: 당신의 이름은 현두예요.
메모리: 이름=현두, 좋아하는 것={'likes': ['김치찜'], 'dislikes': ['해삼']}

[6] 사용자: 내가 좋아하는 것과 싫어하는 것은?
[6] 챗봇: 현두님은 좋아하는 것: 김치찜. 싫어하는 것: 해삼.
메모리: 이름=현두, 좋아하는 것={'likes': ['김치찜'], 'dislikes': ['해삼']}

